In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report, f1_score
import shap
import random
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, accuracy_score,f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout

2023-10-30 21:36:24.538074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-30 21:36:25.093918: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/envs/tf/lib/python3.7/site-packages/nvidia/cudnn/lib:/home/roronoa/.conda/envs/tf/lib/:
2023-10-30 21:36:25.093994: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/roronoa/.conda/

In [2]:
dataset_train = 'Dataset/train'
dataset_test = 'Dataset/test'

In [3]:
img_height, img_width = 150, 150
batch_size = 32

In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize pixel values to [0, 1]
    validation_split=0.2,      # Split dataset into training (80%) and validation (20%)
    rotation_range=20,         # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,     # Randomly shift the width of images
    height_shift_range=0.2,    # Randomly shift the height of images
    shear_range=0.2,           # Apply shear transformations
    zoom_range=0.2,            # Randomly zoom into images
    horizontal_flip=True,      # Randomly flip images horizontally
    fill_mode='nearest'        # Fill empty pixels using the nearest available pixel
)

In [5]:
train_generator = datagen.flow_from_directory(
    dataset_train,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    subset='training'       # Specify 'training' for the training dataset
)

Found 4187 images belonging to 2 classes.


In [10]:
validation_generator = datagen.flow_from_directory(
    dataset_train,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    subset='validation'     # Specify 'validation' for the validation dataset
)

Found 1045 images belonging to 2 classes.


In [11]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(256, activation='relu'),
    Dropout(0.5),  # Add dropout for regularization
    Dense(1, activation='sigmoid')  # Binary classification, so use 'sigmoid' activation
])


In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary classification loss
              metrics=['accuracy'])

In [13]:
epochs = 20  # Adjust the number of epochs as needed
model.fit(train_generator, validation_data=validation_generator, epochs=epochs)

Epoch 1/20


2023-10-30 21:36:46.513210: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8500
2023-10-30 21:36:46.786926: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-10-30 21:36:47.069781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-30 21:36:47.093375: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f73e8744880 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-30 21:36:47.093401: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-10-30 21:36:47.096652: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-30 21:

131/131 [==============================] - 35s 248ms/step - loss: 0.5316 - accuracy: 0.7430 - val_loss: 0.4804 - val_accuracy: 0.7426
Epoch 2/20
131/131 [==============================] - 33s 250ms/step - loss: 0.3570 - accuracy: 0.8290 - val_loss: 0.3249 - val_accuracy: 0.8450
Epoch 3/20
131/131 [==============================] - 33s 251ms/step - loss: 0.3215 - accuracy: 0.8491 - val_loss: 0.2832 - val_accuracy: 0.8584
Epoch 4/20
131/131 [==============================] - 33s 250ms/step - loss: 0.2926 - accuracy: 0.8696 - val_loss: 0.3021 - val_accuracy: 0.8459
Epoch 5/20
131/131 [==============================] - 33s 252ms/step - loss: 0.2932 - accuracy: 0.8706 - val_loss: 0.2874 - val_accuracy: 0.8584
Epoch 6/20
131/131 [==============================] - 33s 250ms/step - loss: 0.2767 - accuracy: 0.8851 - val_loss: 0.2481 - val_accuracy: 0.8785
Epoch 7/20
131/131 [==============================] - 33s 251ms/step - loss: 0.2552 - accuracy: 0.8892 - val_loss: 0.2658 - val_accuracy: 0.8

In [14]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_test,  # Replace with the path to your test dataset directory
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification
    shuffle=False  # Do not shuffle the data for evaluation
)
test_preds = model.predict(test_generator)
predicted_classes = np.round(test_preds).flatten()  # Convert probabilities to binary predictions

true_classes = test_generator.classes

Found 624 images belonging to 2 classes.
20/20 [==============================] - 2s 105ms/step


In [15]:
accuracy = accuracy_score(true_classes, predicted_classes)
clr = classification_report(true_classes, predicted_classes)
f1 = f1_score(true_classes, predicted_classes)
print(f"Accuracy: {accuracy}")
print(f"F1 Score:{f1}")
print(f"Classification Report:\n{clr}")

Accuracy: 0.8990384615384616
F1 Score:0.9232643118148598
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.78      0.85       234
           1       0.88      0.97      0.92       390

    accuracy                           0.90       624
   macro avg       0.91      0.87      0.89       624
weighted avg       0.90      0.90      0.90       624



In [31]:
import shap

# since we have two inputs we pass a list of inputs to the explainer
explainer = shap.GradientExplainer(model, train_generator)

# we explain the model's predictions on the first three samples of the test set
shap_values = explainer.shap_values(test_generator)

AttributeError: 'tuple' object has no attribute 'shape'